In [ ]:
import copy
import os
import warnings

import keras
import matplotlib
import numpy as np
import skimage.io
import skimage.segmentation
import sklearn
import sklearn.metrics
from google.colab import drive
from keras import models
from keras.applications.imagenet_utils import decode_predictions
from keras.models import load_model
from skimage import io
from sklearn.linear_model import LinearRegression

drive.mount("/content/gdrive", force_remount=True)
root_dir = "/content/gdrive/My Drive/"

In [ ]:
# full path
dir_path = os.path.dirname(os.path.realpath(""))

# current dir
cwd = os.getcwd()
cwd

In [ ]:
print("Notebook running: keras ", keras.__version__)
np.random.seed(222)

In [ ]:
matplotlib.use("TkAgg")

io.use_plugin("matplotlib")

%matplotlib inline

Loading the Fine-tuned Inception V3 model. 

In [ ]:
model = load_model("../models/incepv3con.h5")

The instance to be explained (image) is resized and pre-processed to be suitable for Inception V3. This image is saved in the variable Xi.

In [ ]:
Xi = skimage.io.imread("../data/test.jpeg")
Xi = skimage.transform.resize(Xi, (299, 299))
Xi = (Xi - 0.5) * 2  # Inception pre-processing
skimage.io.imshow(Xi / 2 + 0.5)  # Show image before inception preprocessing

Prediction starts...

In [ ]:
np.random.seed(222)
preds = model.predict(Xi[np.newaxis, :, :, :])

In [ ]:
preds

In [ ]:
top_pred_classes = preds[0].argsort()[-2:][::-1]
top_pred_classes

Extract super-pixels from the input image...

In [ ]:
superpixels = skimage.segmentation.quickshift(
    Xi, kernel_size=4, max_dist=200, ratio=0.2
)
num_superpixels = np.unique(superpixels).shape[0]
num_superpixels

In [ ]:
skimage.io.imshow(skimage.segmentation.mark_boundaries(Xi / 2 + 0.5, superpixels))

Creatign random perturbations...

In [ ]:
num_perturb = 150
perturbations = np.random.binomial(1, 0.5, size=(num_perturb, num_superpixels))
perturbations[0]  # Show example of perturbation

The following function perturb_image perturbs the given image (img) based on a perturbation vector (perturbation) and predefined superpixels (segments).

In [ ]:
def perturb_image(img, perturbation, segments):
    active_pixels = np.where(perturbation == 1)[0]
    mask = np.zeros(segments.shape)
    for active in active_pixels:
        mask[segments == active] = 1
    perturbed_image = copy.deepcopy(img)
    perturbed_image = perturbed_image * mask[:, :, np.newaxis]
    return perturbed_image

In [ ]:
skimage.io.imshow(perturb_image(Xi / 2 + 0.5, perturbations[0], superpixels))

Use ML classifier to predict classes of new generated images
This is the most computationally expensive step in LIME because a prediction for each perturbed image is computed. From the shape of the predictions we can see for each of the perturbations we have the output probability for each of the existing class in Inception V3.

In [ ]:
predictions = []
for pert in perturbations:
    perturbed_img = perturb_image(Xi, pert, superpixels)
    pred = model.predict(perturbed_img[np.newaxis, :, :, :])
    predictions.append(pred)

predictions = np.array(predictions)
predictions.shape

Compute distances between the original image and each of the perturbed images and compute weights (importance) of each perturbed image
The distance between each randomly generated perturnation and the image being explained is computed using the cosine distance. For the shape of the distances array it can be noted that, as expected, there is a distance for every generated perturbation.

In [ ]:
original_image = np.ones(num_superpixels)[
    np.newaxis, :
]  # Perturbation with all superpixels enabled
distances = sklearn.metrics.pairwise_distances(
    perturbations, original_image, metric="cosine"
).ravel()
distances.shape

Use kernel function to compute weights

In [ ]:
kernel_width = 0.25
weights = np.sqrt(np.exp(-(distances**2) / kernel_width**2))  # Kernel function
weights.shape

Use perturbations, predictions and weights to fit an explainable (linear) model

In [ ]:
class_to_explain = top_pred_classes[0]
simpler_model = LinearRegression()
simpler_model.fit(
    X=perturbations, y=predictions[:, :, class_to_explain], sample_weight=weights
)
coeff = simpler_model.coef_[0]
coeff

Compute top features (superpixels)

In [ ]:
num_top_features = 25
top_features = np.argsort(coeff)[-num_top_features:]
top_features

Show LIME explanation (image with top features)

In [ ]:
mask = np.zeros(num_superpixels)
mask[top_features] = True  # Activate top superpixels
skimage.io.imshow(perturb_image(Xi / 2 + 0.5, mask, superpixels))

The end.